In [11]:
import numpy as np
import napari
from napari.types import ImageData
from magicgui import FunctionGui
from functools import wraps
from skimage.filters import gaussian
from skimage.io import imread, imsave
from inspect import Signature, signature
from functools import partial

def process_image(image: ImageData, sigma=3) -> ImageData:
    return gaussian(image, sigma=sigma)

class MyGui(FunctionGui):
    def __init__(self,function,name):
        super().__init__(
          function,
          call_button=False,
          layout='vertical',
          auto_call=True,
          name = name
          )
def signature_w_kwargs_from_function(function, arg_vals: list):
    """
    Returns a new signature for a function in which the default values are replaced
    with the arguments given in arg_vals

    Parameters
    ----------
    function: 
        input function to generate new signature

    arg_vals: list
        list of arguments to replace defaults in signature
    """

    # getting the keywords corresponding to the values
    keyword_list = list(signature(function).parameters.keys())

    # creating the kwargs dict
    kw_dict = {}
    for kw, val in zip(keyword_list, arg_vals):
        kw_dict[kw] = val
        
    possible_input_image_names = ['image',
                                  'label_image',
                                  'binary_image'
                                 ]
    for name in possible_input_image_names:
        try:
            kw_dict.pop(name) # we are making an assumption that the input will aways be this
        except KeyError:
            pass

    return signature(partial(function, **kw_dict))

def make_gui(func, viewer, name):
    """
    Function returns a widget with a GUI for the function provided in the parameters,
    that can be added to the napari viewer. Largely copied from @haesleinhuepf (I can't remember where though)
    

    Parameters
    ----------
    func: 
        input function to generate the gui for
    viewer:
        napari.Viewer instance to which the widget is added
    
    autocall: Boolean
        set wether the function is automatically called when a parameter is changed
    """
    gui = None

    from napari.types import ImageData, LabelsData
    import inspect
    sig = inspect.signature(func)

    @wraps(func)
    def worker_func(*iargs, **ikwargs):
        data = func(*iargs, **ikwargs)
        if data is None:
            return None

        target_layer = None

        if sig.return_annotation in [ImageData, "napari.types.ImageData", LabelsData, "napari.types.LabelsData"]:
            op_name = name
            new_name = f"{op_name} result"

            # we now search for a layer that has -this- magicgui attached to it
            try:
                # look for an existing layer
                target_layer = next(x for x in viewer.layers if x.source.widget is gui)
                target_layer.data = data
                target_layer.name = new_name
                # layer.translate = translate
            except StopIteration:
                # otherwise create a new one
                from napari.layers._source import layer_source
                with layer_source(widget=gui):
                    if sig.return_annotation in [ImageData, "napari.types.ImageData"]:
                        target_layer = viewer.add_image(data, name=new_name)
                    elif sig.return_annotation in [LabelsData, "napari.types.LabelsData"]:
                        target_layer = viewer.add_labels(data, name=new_name)

        if target_layer is not None:
            # update the workflow manager in case it's installed
            try:
                from napari_workflows import WorkflowManager
                workflow_manager = WorkflowManager.install(viewer)
                workflow_manager.update(target_layer, func, *iargs, **ikwargs)
            except ImportError:
                pass

            return None
        else:
            return data

    gui = MyGui(worker_func, name)
    return gui

image1 = np.zeros((21,21))
image1[10,10] = 255

image2 = imread('blobs.tif')

image3 = (np.random.rand(21,21)*255).astype('uint8')
imsave('random8bitint.tif', image3)
image3_loaded_as_8bit_int = imread('random8bitint.tif')
loaded_to_float = (image3_loaded_as_8bit_int/255).astype('float64')
viewer = napari.Viewer()
#viewer.add_image(image1)
#viewer.add_image(image2)
viewer.add_image(image3)
viewer.add_image(loaded_to_float)
names = [f'function {i}' for i in range(1,6)]
for i,name in enumerate(names):
    func = process_image
    sig = signature_w_kwargs_from_function(func, ('image2',i+3))
    gui = make_gui(func, viewer, name)
    viewer.window.add_dock_widget(gui, name = name)

C:\Users\ryans\AppData\Local\Temp\ipykernel_5816\695812661.py:4: DeprecationWarning: magicgui.FunctionGui is deprecated. Please import at magicgui.widgets.FunctionGui
  from magicgui import FunctionGui
C:\Users\ryans\anaconda3\envs\np_workflows_v1\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [5]:
gaussian?

In [12]:
loaded_to_float == viewer.layers['function 1 result'].data

array([[False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False,  True,  True, False,
         True,  True,  True],
       [ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False,  True,  True,  True,  True,  True,  True,
         True, False,  True],
       [ True,  True,  True,  True,  True,  True,  True, False,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True